In [8]:
!pip install memory_profiler
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [9]:
import pandas as pd, os, csv, time, random
from collections import defaultdict

# seed bisa random setiap run
RANDOM_SEED = int(time.time()) % 100000
random.seed(RANDOM_SEED)
print("🎲 Random seed:", RANDOM_SEED)

# parameter dasar
CSV_PATH = "/content/sample_data/jadwal_kuliah_random_fix9_halfhour.csv"
N_STUDENTS = 300

REQUIRED = {"EF234501","EF234502","EF234503","EF234504","EF234505"}
ELECTIVES = {"EF234509","EF234518","EF234713","EK234601","EF234618",
             "EF234606","ER234504","EF234614","EF234704","EF234708"}


🎲 Random seed: 89537


In [10]:
def to_minutes(hhmm):
    h, m = map(int, hhmm.split(":"))
    return h*60 + m

def conflict(a,b):
    return a["day"]==b["day"] and not (a["end"]<=b["start"] or b["end"]<=a["start"])

def load_schedule(path):
    data={}
    with open(path,newline="",encoding="utf-8") as f:
        reader=csv.DictReader(f)
        for r in reader:
            code=r["Kode Kelas"].split()[0]
            cap=60 if code in REQUIRED else 30
            data[r["Kode Kelas"]]={"code":code,"name":r["Nama Mata Kuliah"],
                                   "day":r["Hari"],
                                   "start":to_minutes(r["Mulai"]),
                                   "end":to_minutes(r["Selesai"]),
                                   "cap":cap}
    return data

def group_by_course(sections):
    g=defaultdict(list)
    for sid,s in sections.items():
        g[s["code"]].append(sid)
    return g


In [11]:
sections = load_schedule(CSV_PATH)
course_to_secs = group_by_course(sections)
capacity = {sid:s["cap"] for sid,s in sections.items()}

print("Total section:", len(sections))
print("Wajib seats:", sum(s['cap'] for s in sections.values() if s['code'] in REQUIRED))
print("Pilihan seats:", sum(s['cap'] for s in sections.values() if s['code'] in ELECTIVES))


Total section: 45
Wajib seats: 1500
Pilihan seats: 600


In [12]:
def greedy_assign_student(course_to_secs, sections, capacity):
    assigned = {}
    assigned_slots = defaultdict(list)
    current_capacity = capacity.copy()

    # matkul wajib dulu
    courses_to_assign = list(REQUIRED)
    electives_to_choose = random.sample(list(ELECTIVES), 2)
    courses_to_assign.extend(electives_to_choose)
    random.shuffle(courses_to_assign[len(REQUIRED):])


    for course_code in courses_to_assign:
        secs = sorted(course_to_secs[course_code], key=lambda sid: (current_capacity[sid], sections[sid]["start"]), reverse=True)

        picked = False
        for sid in secs:
            s = sections[sid]

            conflicts_on_day = False
            for assigned_start, assigned_end in assigned_slots[s["day"]]:
                if conflict({"day": s["day"], "start": s["start"], "end": s["end"]},
                            {"day": s["day"], "start": assigned_start, "end": assigned_end}):
                    conflicts_on_day = True
                    break

            if conflicts_on_day:
                continue

            if current_capacity[sid] > 0:
                assigned[course_code] = sid
                assigned_slots[s["day"]].append((s["start"], s["end"]))
                assigned_slots[s["day"]].sort()
                current_capacity[sid] -= 1
                picked = True
                break

        if not picked:
            # jika matkul wajib tidak bisa, jadwal invalid
            if course_code in REQUIRED:
                 return None
            pass

    # cek semua matkul wajib terpilih
    if all(c in assigned for c in REQUIRED):
        return assigned
    else:
        return None



In [13]:
%%memit

out_dir="/content/output_greedy"
os.makedirs(out_dir,exist_ok=True)

start=time.time()
success=0
for i in range(1,N_STUDENTS+1):
    sid=f"S{i:03d}"
    alloc=greedy_assign_student(course_to_secs,sections,capacity)
    if not alloc:
        print(f"[!] {sid} gagal alokasi")
        continue

    # simpan jadwal mahasiswa
    day_order={"Senin":1,"Selasa":2,"Rabu":3,"Kamis":4,"Jumat":5}
    rows=[]
    for c,sec in alloc.items():
        info=sections[sec]
        jenis="Wajib" if c in REQUIRED else "Pilihan"
        rows.append([info["day"],
                     f"{info['start']//60:02d}:{info['start']%60:02d}",
                     f"{info['end']//60:02d}:{info['end']%60:02d}",
                     sec, info["name"], jenis])
    rows.sort(key=lambda r:(day_order[r[0]],r[1]))
    with open(os.path.join(out_dir,f"{sid}.csv"),"w",encoding="utf-8") as f:
        f.write("Hari,Mulai,Selesai,Kode Kelas,Nama Mata Kuliah,Jenis\n")
        for r in rows: f.write(",".join(r)+"\n")
    success+=1
    if i%25==0: print(f"Progress: {i}/300 (sukses: {success})")

elapsed=time.time()-start
print(f"\n✅ Done. Mahasiswa sukses: {success}/{N_STUDENTS} | Waktu: {elapsed:.2f}s")


Progress: 25/300 (sukses: 25)
Progress: 50/300 (sukses: 50)
Progress: 75/300 (sukses: 75)
Progress: 100/300 (sukses: 100)
Progress: 125/300 (sukses: 125)
Progress: 150/300 (sukses: 150)
Progress: 175/300 (sukses: 175)
Progress: 200/300 (sukses: 200)
Progress: 225/300 (sukses: 225)
Progress: 250/300 (sukses: 250)
Progress: 275/300 (sukses: 275)
Progress: 300/300 (sukses: 300)

✅ Done. Mahasiswa sukses: 300/300 | Waktu: 0.06s
peak memory: 174.84 MiB, increment: 0.00 MiB


In [14]:
# import zipfile
# zip_path="/content/jadwal_greedy_300.zip"
# with zipfile.ZipFile(zip_path,"w",zipfile.ZIP_DEFLATED) as zf:
#     for fn in sorted(os.listdir(out_dir)):
#         zf.write(os.path.join(out_dir,fn),arcname=fn)
# print("ZIP siap:", zip_path)

# from google.colab import files
# files.download(zip_path)
